In [1]:
import gym
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','content'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self,inputs,outputs):
        super(DQN, self).__init__()
        self.conn1 = nn.Linear(inputs,32)
        self.conn2 = nn.Linear(32,128)
        self.conn3 = nn.Linear(128,16)
        self.conn4 = nn.Linear(16, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float().to(device)
        x = F.relu(self.conn1(x))
        x = F.relu(self.conn2(x))
        x = F.relu(self.conn3(x))
        return self.conn4(x)

    

In [4]:
data_path = '/home/ubuntu/data/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv')
UIT

,u,i,day,time,viewtime,video_type,video_format,city,city_isp,client_ip,conn_type,device_type
0,365,3391,0,0,148,11,21,0,0,8812,1,0
1,203,5779,0,0,7,11,4,0,0,9063,1,0
2,208,4675,0,0,92,13,4,0,0,3444,1,0
3,159,332,0,0,56,11,3,0,0,4058,1,0
4,50,674,0,0,439,11,4,0,0,1549,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,29,2591880,34,11,4,0,0,7061,1,0
300979,158,8448,29,2591880,34,11,4,0,0,11316,1,0
300980,483,6463,29,2591940,35,11,4,0,0,7061,1,0
300981,158,4715,29,2591940,34,11,4,0,0,11316,1,0


In [5]:
trainUIT = UIT[UIT['day']<18]
validUIT = UIT[UIT['day']<24]
contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())
contentNum,userNum,trainUIT,validUIT

(10000,
 500,
           u     i  day     time  viewtime  video_type  video_format  city  \
 0       365  3391    0        0       148          11            21     0   
 1       203  5779    0        0         7          11             4     0   
 2       208  4675    0        0        92          13             4     0   
 3       159   332    0        0        56          11             3     0   
 4        50   674    0        0       439          11             4     0   
 ...     ...   ...  ...      ...       ...         ...           ...   ...   
 198170  264  7442   17  1555140        90          13             4     0   
 198171   19  9362   17  1555140       424          13             4     0   
 198172   82  9223   17  1555140        94          15             4     0   
 198173   35  4164   17  1555140        22          11             4     0   
 198174  239  5062   17  1555140        89          13             4     0   
 
         city_isp  client_ip  conn_type  device_

In [6]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
 
        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p), 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                torch.from_numpy(self.l))

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()


In [7]:
class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S, self.l = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.statusFeature = self.statusEmbedding()

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1


    def statusEmbedding(self):
        statusFeature = torch.zeros(size=(5,env.contentNum)).to(device)
        
        statusFeature[0] = self.v
        statusFeature[1] = self.r[self.u]
        statusFeature[2] = self.p
        statusFeature[3] = self.e
        statusFeature[4] = self.S

        #statusFeature[5] = status['r']
        return statusFeature.T
    
    def getReward(self,lastru,lastp,ru,p,i,action,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (torch.log(v * p + (1-v) * (1-p)).sum() / torch.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh =   self.ALPHAh * (torch.log(ru * p + (1-ru) * (1-p)).sum() - torch.log(v * p + (1-v) * (1-p)).sum())
        self.Rh = self.ALPHAh * ( 1 / ( 1 + torch.exp( torch.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        #self.Rh =   self.ALPHAh * torch.log(ru * p + (1-ru) * (1-p))

        self.Ro =   self.BETAo * action[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * action[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - action[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  self.Rh + self.Ro + self.Rl

    def selectAction(self,env,uit,QNetwork,train,memory):
        
        self.updateViewContent(uit[1])
        
        self.lastStatusFeature = self.statusFeature
        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.r , self.p , self.e, self.S, self.l = env.getStatus()
        self.statusFeature = self.statusEmbedding()
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.l,self.e,self.v)

        if self.W[-1] not in np.argwhere(self.lastAction.numpy()): #local cache miss
            sample = random.random()
            eps_threshold = EPS_END + (EPS_START - EPS_END) *  np.exp(-1. * agentStep / EPS_DECAY)
            if  not train or (train and sample > eps_threshold): #var greedy
                QNetwork.eval()
                with torch.no_grad():
                    Q_value = QNetwork(self.statusFeature)
                    actionIndex = list((Q_value[:,1]-Q_value[:,0]).argsort(descending=True)[0:self.Bu])
                QNetwork.train()
            else:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))

            self.action = torch.zeros(size=(env.contentNum,),dtype=int)
            self.action[self.W[-1]] = 1
            if self.W[-1] not in actionIndex:
                actionIndex.pop()
            for index in actionIndex:
                self.action[index] = 1
            env.updateEnv(self.u,self.action.numpy(),uit[2])
        else:
            self.action = self.lastAction # keep the cache and no request the new video

        if train:
            lastAction = torch.cat(((1-self.lastAction).unsqueeze(1),self.lastAction.unsqueeze(1)),1)
            memory.push(self.lastStatusFeature, 
                        lastAction.to(device), 
                        self.statusFeature,
                        torch.tensor([self.reward.float()]).to(device),
                        torch.tensor([self.W[-1]]).to(device))
                        
        return self.action

In [8]:
BATCH_SIZE = 128
GAMMA = 0.99

def optimize_model():
    if len(memory) < BATCH_SIZE:
        #print(len(memory))
        return 0
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    #non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    #non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    content_batch = torch.cat(batch.content)
    next_state_batch = torch.cat(batch.next_state)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_mask_bacth = action_batch.ge(0.5)
    state_action_values = torch.stack(torch.masked_select(policy_net(state_batch),state_action_mask_bacth).chunk(BATCH_SIZE,dim=0)).sum(dim=1)
    #print(state_action_values,state_action_values.dtype)
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    def getNextStatusQ(s_batch,c_batch):
        
        Q_value= torch.stack(target_net(s_batch).detach().chunk(BATCH_SIZE,dim=0))
        c = c_batch.chunk(BATCH_SIZE,dim=0)
        action = torch.zeros(size=(BATCH_SIZE,contentNum,2),dtype=int).to(device)
        for b in range(BATCH_SIZE):
            Q_value_sortindex = list((Q_value[b,:,1]-Q_value[b,:,0]).argsort(descending=True)[0:Bu])
            i = c[b].squeeze()
            if i not in Q_value_sortindex:
                Q_value_sortindex.pop()
            action[b,i,1] = 1
            for index in Q_value_sortindex:
                action[b,index,1] = 1
        action[:,:,0]=1-action[:,:,1]
        action_mask = action.ge(0.5).to(device)
        next_state_values = torch.stack(torch.masked_select(Q_value,action_mask).chunk(BATCH_SIZE,dim=0)).sum(dim=1).float()

        return next_state_values
    
    next_state_values =  getNextStatusQ(next_state_batch,content_batch)

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values.unsqueeze(1), expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

    return loss

In [9]:
import os
pathName = '/home/ubuntu/data/PEC/model_dict/R3009_U5_V100/'
if not os.path.exists(pathName):
    os.makedirs(pathName)
MODELPATH = pathName+'dnn_h1_exp_'

rewardPara = {"alpha":1,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 15

num_episodes = 10
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = trainUIT.shape[0]*3
agentStep = 0

In [10]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(5, 2).to(device)
target_net = DQN(5, 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)
torch.cuda.empty_cache()

bestReward =  float("-inf")

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)
        agentStep += 1

        # train the DQN network
        if index % 25 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        if index % 100 == 0 :
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())
            if index % 50000 == 0 :
                psi = 0
                p = torch.from_numpy(env.p)
                for u in UEs:
                    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
                print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
                print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
                print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
                print()
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------train----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------train----------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    
    #valid the hyperparameters
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in validUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,0,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)

        if index % 50000 == 0 :
            psi = 0
            p = torch.from_numpy(env.p)
            for u in UEs:
                psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
            print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
            print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
            print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
            print()
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------validation----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------validation----------------------------------")
    print()
    modelSavePath = MODELPATH+'ep{}_'.format(i_episode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
    torch.save(policy_net.state_dict(),modelSavePath)
    if sumReward.sum() > bestReward:
        bestReward = sumReward.sum()
        bestEpisode = i_episode
        bestPath = modelSavePath
        print()
        print(bestPath)
        print()

--Time: Sat Oct  9 14:16:12 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0.5 0.  0. ] total reward: 0.5
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Sat Oct  9 14:23:46 2021 Episode: 0   Index: 50000   Loss: 0.31138 --
Reward: [0.68656 0.00499 0.0688 ] total reward: 0.76035
UEHitrate: 0.00608  edgeHitrate 0.086 sumHitrate 0.09208  privacy: 2.59524

--Time: Sat Oct  9 14:34:46 2021 Episode: 0   Index: 100000   Loss: 0.28896 --
Reward: [0.63533 0.00475 0.06412] total reward: 0.7042
UEHitrate: 0.00567  edgeHitrate 0.08015 sumHitrate 0.08582  privacy: 1.92335

--Time: Sat Oct  9 14:45:57 2021 Episode: 0   Index: 150000   Loss: 0.27445 --
Reward: [0.60361 0.00517 0.06272] total reward: 0.6715
UEHitrate: 0.00618  edgeHitrate 0.07841 sumHitrate 0.08459  privacy: 1.49094

------------------------------train----------------------------------
--Time: Sat Oct  9 14:57:01 2021 Episode: 0   Index: 198174   Loss: 0.26308 --
Reward: [0.5851  0.00546 0.06255] total r

In [11]:
bestPath

'/home/ubuntu/data/PEC/model_dict/R3009_U5_V100/dnn_h1_exp_ep0_1009-15-23-37'

In [12]:
#test

policy_net = DQN(5, 2).to(device)
target_net = DQN(5, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
torch.cuda.empty_cache()


bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()

print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 10 03:03:22 2021   Index: 0   Loss: 0.0 --
Reward: [0.5 0.  0. ] total reward: 0.5
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Sun Oct 10 03:04:30 2021   Index: 10000   Loss: 0.0 --
Reward: [0.85502 0.00318 0.24862] total reward: 1.10681
UEHitrate: 0.0095  edgeHitrate 0.30987 sumHitrate 0.31937  privacy: 2.60748

--Time: Sun Oct 10 03:05:34 2021   Index: 20000   Loss: 0.0 --
Reward: [0.83995 0.00362 0.24119] total reward: 1.08475
UEHitrate: 0.00725  edgeHitrate 0.30213 sumHitrate 0.30938  privacy: 2.35088

--Time: Sun Oct 10 03:06:41 2021   Index: 30000   Loss: 0.0 --
Reward: [0.83076 0.00395 0.21845] total reward: 1.05316
UEHitrate: 0.00697  edgeHitrate 0.27399 sumHitrate 0.28096  privacy: 2.22801

--Time: Sun Oct 10 03:07:55 2021   Index: 40000   Loss: 0.0 --
Reward: [0.83003 0.00399 0.20907] total reward: 1.04309
UEHitrate: 0.00677  edgeHitrate 0.26194 sumHitrate 0.26872  privacy: 2.11675

--Time: Sun Oct 10 03:09:03 2021   Index: 50000 

In [13]:
bestPath

'/home/ubuntu/data/PEC/model_dict/R3009_U5_V100/dnn_h1_exp_ep0_1009-15-23-37'

In [14]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.31937, 0.30938, 0.28096, 0.26872, 0.26195, 0.25816,
        0.25104, 0.24861, 0.24464, 0.24177, 0.24023, 0.23554, 0.23211,
        0.23271, 0.23295, 0.23062, 0.23151, 0.23289, 0.23263, 0.23123,
        0.23199, 0.23082, 0.22964, 0.22842, 0.22817, 0.22694, 0.22561,
        0.22571, 0.22794, 0.23081]),
 array([0.     , 0.0095 , 0.00725, 0.00697, 0.00677, 0.00664, 0.00635,
        0.00599, 0.00585, 0.00571, 0.00554, 0.00538, 0.00528, 0.00535,
        0.00539, 0.00539, 0.00539, 0.00534, 0.00522, 0.00524, 0.00513,
        0.00512, 0.00507, 0.00505, 0.00505, 0.00506, 0.00507, 0.00507,
        0.00504, 0.00502, 0.00499]),
 array([0.     , 0.30987, 0.30213, 0.27399, 0.26194, 0.25531, 0.25181,
        0.24505, 0.24276, 0.23893, 0.23623, 0.23484, 0.23026, 0.22677,
        0.22731, 0.22756, 0.22523, 0.22618, 0.22767, 0.22739, 0.22609,
        0.22687, 0.22575, 0.22459, 0.22337, 0.22311, 0.22188, 0.22054,
        0.22066, 0.22293, 0.22583]))

In [15]:
privacyReduction

array([4.02118, 2.60748, 2.35088, 2.22801, 2.11675, 2.02317, 1.92269,
       1.85717, 1.76463, 1.67848, 1.5724 , 1.48494, 1.43214, 1.37793,
       1.32209, 1.27118, 1.20669, 1.1508 , 1.10139, 1.04912, 1.00822,
       0.95159, 0.90192, 0.85207, 0.80474, 0.76386, 0.72363, 0.69054,
       0.65739, 0.62339, 0.59651])

In [16]:
bestPath

'/home/ubuntu/data/PEC/model_dict/R3009_U5_V100/dnn_h1_exp_ep0_1009-15-23-37'